# use guanjia scoring API to score txn

In [ ]:
from PyGuanjia import GuanjiaClient
from PyGuanjia import ModelCategory, ModelDomain, ModelMO, ModelStatus, ModelType
from model_automation.utils.rmr import run_cmd

from credentials import PAZ_PASSWORD



data_dir = 'hdfs://horton/apps/risk/det/guxia/ucc_latam23/data/training/ucc_latam_cbp_oot_20221101_20230131/20230214/madmen_pig'
header_path = 'hdfs://horton/apps/risk/det/guxia/ucc_latam23/data/training/ucc_latam_cbp_oot_20221101_20230131/20230214/madmen_pig/.pig_header'     
model_id = '62e93905f4a09f8a94c70897'
record_columns = ['driver_trans_id', 'driver_monthly']

username = ''

In [2]:
guanjia = GuanjiaClient(username=username, isQA=False, isPAZ=True)
guanjia.login(passwd=PAZ_PASSWORD)


In [3]:
job = guanjia.scoring(
    modelId=model_id,
    csvPath=data_dir,
    headerPath=header_path,
    delimiter='^G',
    recordCols='|'.join(record_columns),
    isMultiple=True,
)
print(f"guanjian job id: {job['jobId']}")

In [ ]:
while guanjia.get_job_status(jobId=job['jobId']) == 'RUNNING':
    print("scoring is running ...")
    scoring_file_num = os.popen(f"hadoop fs -ls /apps/risk/det/guanjia/score_result/{job['jobId']}* | grep part | wc -l").read().strip()

    if scoring_file_num == input_file_num:
        print(f"scoring output already finished, please check /apps/risk/det/guanjia/score_result/{job['jobId']}*")
        time.sleep(30)
        break
    time.sleep(30)

if guanjia.get_job_status(jobId=job['jobId']) == 'COMPLETE_SUCC':
    print("scoring finished")
    score_path = guanjia.get_job_details(jobId=job['jobId'])['Result Path in HDFS']
    score_list = guanjia.get_job_details(jobId=job['jobId'])['Score Name'].split(', ')
elif scoring_file_num == input_file_num:
    path = run_cmd(f"hadoop fs -ls /apps/risk/det/guanjia/score_result/{job['jobId']}/* | grep 'part' | head -1")
    score_path = path.split(' ')[-1].rsplit('/', 1)[0]
    score_list = []
    for model_id in [conf.model_id]:
        modelInfo = guanjia.get_model_info(modelId=model_id)
        if modelInfo['placeholderName'] is not None and modelInfo['placeholderName'] != '':
            score_list.append(modelInfo['placeholderName'] + '_model_score1')
        else:
            score_list.append('_'.join([modelInfo['name'], modelInfo['modelVersion'], 'score']))
else:
    job_detail = guanjia.get_job_details(jobId=job['jobId'])
    print(json.dumps(job_detail, indent=4))
    raise Exception(f"guanjia scoring job {job['jobId']} failed!")
    
print("output data path : " + score_path)
print("output score list : " + ",".join(score_list))


# guanjia scoring: v2

- donot put model score name in recordCols
- after job sbumitted, to guanjia webui and check job page

In [ ]:
from PyGuanjia import GuanjiaClient
from PyGuanjia import ModelCategory, ModelDomain, ModelMO, ModelStatus, ModelType
from model_automation.utils.rmr import run_cmd

from credentials import GUANJIA_PASSWORD


model_ids = []
data_dir = ''
meta_columns = []

In [ ]:
guanjia = GuanjiaClient(username=username, isQA=False, isPAZ=True)
guanjia.login(passwd=PAZ_PASSWORD)


job = guanjia.scoring(
    modelId="|".join(model_ids),
    csvPath=data_dir,
    headerPath=os.path.join(data_dir, '.pig_header'),
    delimiter='^G',
    recordCols="|".join(meta_columns),
)

# use guanjia to download model ume

In [ ]:
from PyGuanjia import GuanjiaClient
from pyScoring import UMEModel

import credentials
importlib.reload(credentials)
from credentials import PAZ_PASSWORD


model_ids = {
    'UCC23': '16a108931cc54a7b95ca625',
}

model_dir = 'guanjia_model_dir'

In [ ]:
guanjia = GuanjiaClient(username=username, isQA=False, isPAZ=True)
guanjia.login(passwd=PAZ_PASSWORD)

for model_name, model_id in model_ids.items():
    print(model_name, model_id)
    ret_model_dir = f'{model_dir}_{model_name}'
    if os.path.exists(ret_model_dir):
        print(1)
        shutil.rmtree(ret_model_dir)
        
    guanjia.download_model(modelId=model_id,
                           folderName=ret_model_dir)



# upload ume to guanjia

In [ ]:
from PyGuanjia import GuanjiaClient
from PyGuanjia import ModelCategory, ModelDomain, ModelMO, ModelStatus, ModelType
from credentials import PAZ_PASSWORD


username = ''
ume_path = '' # path to ume file
model_name = '' # if you use existing model name, guanjia will overwrite it use the ume you provide
extra_owner = ['wzhao5', 'siwan']
model_seg = 'UCC_SF'


In [ ]:
# upload ume to guanjia

guanjia = GuanjiaClient(username=username, isQA=False, isPAZ=True)
guanjia.login(passwd=PAZ_PASSWORD)

model_id = guanjia.create_model(modelName=model_name,
                                MO=model_seg,
                                spec=ume_path,
                                modelCategory=ModelCategory.NON_ISO,
                                modelType=ModelType.M,
                                SEG=model_seg,
                                extraOwners=extra_owner,
                                domain=ModelDomain.RISK,
                                targetService="RUCS",
                                description=model_name,
                               )

print('model id', model_id)

# validate guanjia ume

In [ ]:
# preppare a guanjia validation file.
validation_file_path = ''
guanjia_valid_data_path = ''
ume_path = ''
ume_output_score_name = ''

# columns you want guanjia to keep in return result
record_columns = ['driver_trans_id', 'final_score', 'cast_normed_score']

In [ ]:
# When loading csv format data, do remember to set dtype=str, or ume prediction result will be different.

valid_df = pd.read_csv(validation_file_path, dtype=str, sep='\x07', keep_default_na=False)

ume = UMEModel(ume_path)
valid_ret = ume.predict_pandas(valid_df)
valid_ret.head()
valid_ret.to_csv(guanjia_valid_data_path, sep='\x07', index=False)


job_id = guanjia.validate(
    modelId=model_id,
    csv=guanjia_valid_data_path,
    delimiter='\x07',
    expectScoreName=ume_output_score_name,
    recordCols=record_columns,
    threshold=1.0,
    precision=0.1
)

print('validation job id', job_id)


# get RMR champion model info

In [ ]:
from PyGuanjia import GuanjiaClient
from credentials import PAZ_PASSWORD


username = ''
mo_name = 'UCC_SF'

In [ ]:
# upload ume to guanjia
guanjia = GuanjiaClient(username=username, isQA=False)
guanjia.login(passwd=PAZ_PASSWORD)
champion_model_info = guanjia.get_champion_model(mo=mo_name)
print(champion_model_info)